In [14]:
%matplotlib inline
from azureml.core import Workspace
import numpy as np
import matplotlib.pyplot as plt
import azureml.core
import configparser

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.21.0


In [15]:
config = configparser.ConfigParser()
config.read('episode_vii.ini')
ws = Workspace.get(name=config['azure.settings']['workspace_name'], 
                   subscription_id=config['azure.settings']['subscription_id'], 
                   resource_group=config['azure.settings']['resource_group'])
print('Workspace loaded.')

Workspace loaded.


In [7]:
from azureml.core import Experiment
experiment_name = 'Simple-Regression-Experiment'
exp = Experiment(workspace=ws, name=experiment_name)

In [12]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

compute_name = 'franklai-basic-compute'
compute_target = ws.compute_targets[compute_name]
print(type(compute_target))
print('Connected to compute ' + compute_name + '.')

<class 'azureml.core.compute.computeinstance.ComputeInstance'>
Connected to compute franklai-basic-compute.


In [17]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import uuid

storage_connection_string = config['azure.settings']['storage_connection_string']
blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
print('Blob service client created.')

container_client = None
for container in blob_service_client.list_containers():
    if container.name == config['azure.settings']['blob_container_name']:
        container_client = blob_service_client.get_container_client(container)
        print('Got the container client.')

if container_client == None:
    print('No container client, creating one with a UUID as the name.')
    # Create a unique name for the container
    container_name = str(uuid.uuid4())

    # Create the container
    container_client = blob_service_client.create_container(container_name)
    print('Container created:', container_name)

Blob service client created.


In [ ]:
csv_file_name = '/data/regression.csv'

csv_file = open(csv_file_name, 'rb')
container_client.upload_blob(name='regression.csv', data=csv_file)
print('Blob upload complete.')

In [20]:
from azure.storage.blob import BlobProperties

download_file = './data/regression_download.csv'
with open(download_file, "wb") as my_blob:
    download_stream = container_client.download_blob(blob=BlobProperties(name='regression.csv'))
    my_blob.write(download_stream.readall())
    print('Blob download complete.')

Blob download complete.


# References

* https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-train-models-with-aml